In [29]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import requests
import pandas as pd
import time
from datetime import datetime

# --- PARÁMETROS ESPECÍFICOS PARA TU TAREA ---
GENRES = ['reggaeton']
YEARS = [2000, 2024] 

In [3]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [34]:
import os
from dotenv import load_dotenv

load_dotenv() # Carga las variables del archivo .env

SPOTIPY_CLIENT_ID = os.getenv('SPOTIPY_CLIENT_ID')
SPOTIPY_CLIENT_SECRET = os.getenv('SPOTIPY_CLIENT_SECRET')
LASTFM_API_KEY = os.getenv('LASTFM_API_KEY')


In [51]:
# Años para iterar
YEARS = [2000, 2024] 

# Términos optimizados para una búsqueda de ALTO VOLUMEN
# Incluye etiquetas históricas ('dembow') y modernas ('trap latino', 'urbano latino')
query_terms = [
    'reggaeton',
    'latin urban',
    'pop latino',
    'trap latino',
    'urbano latino',
    'dembow',
    '"hip hop latino"' # Entre comillas para buscar la frase exacta (útil para el 2000)
]

# 4. Inicializa el cliente de Spotify
try:
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
        client_id=SPOTIPY_CLIENT_ID,
        client_secret=SPOTIPY_CLIENT_SECRET
    ))
    print("✅ Conexión a Spotify exitosa.")
except Exception as e:
    print(f"❌ Error de conexión a Spotify: {e}")
    sp = None

✅ Conexión a Spotify exitosa.


In [47]:
# NUEVA CELDA: Comprobación de autenticación
print(type(sp)) 

<class 'spotipy.client.Spotify'>


In [52]:
# --- DEFINICIÓN DE LA FUNCIÓN CON DOBLE BUCLE (MÁXIMA ROBUSTEZ) ---

def extract_spotify_data_reggaeton_ampliada(sp, years, query_terms, max_total_songs=2000): 
    """
    VERSION FINAL ROBUSTA: Itera primero por AÑO y luego por CADA TÉRMINO de género.
    Esto garantiza que la API acepte todas las consultas y maximiza los resultados.
    """
    
    spotify_data = []
    
    for year in years:
        year_range = f'{year}-{year}'
        print(f"\n--- INICIANDO EXTRACCIÓN PARA EL AÑO {year} ---")

        # BUENO 1: Iteramos sobre cada término de búsqueda (reggaeton, trap latino, etc.)
        for term in query_terms:
            
            # Construcción de la consulta simple (solo 1 término)
            query = f'{term} year:{year_range}'
            
            print(f"   -> Buscando con término: {term}")
            
            try:
                # Hacemos la primera llamada
                results = sp.search(q=query, type='track', limit=50) 
                
                total_resultados_api = results['tracks']['total'] if results and 'tracks' in results else 0
                print(f"   -> Spotify reporta {total_resultados_api} resultados para '{term}'.")
                
                songs_added_in_term = 0
                
                # BUENO 2: Bucle de Paginación
                while results and songs_added_in_term < max_total_songs:
                    
                    for track in results['tracks']['items']:
                        
                        # Usamos un conjunto de IDs para evitar duplicados si un artista aparece en dos géneros
                        track_id = track['id']
                        
                        # Si ya tenemos esta canción de este año, la saltamos
                        if any(d.get('ID_Spotify') == track_id for d in spotify_data):
                            continue
                            
                        if songs_added_in_term >= max_total_songs:
                            break
                        
                        data_row = {
                            'Artista': track['artists'][0]['name'],
                            'Género musical': f'Reggaeton (Origen: {term})', # Indicamos el término que la encontró
                            'Tipo': 'Canción',
                            'Nombre': track['name'],
                            'Año de lanzamiento': year,
                            'ID_Spotify': track_id,
                            'ID_Album': track['album']['id']
                        }
                        spotify_data.append(data_row)
                        songs_added_in_term += 1

                    # Paginación
                    if results['tracks']['next']:
                        results = sp.next(results['tracks'])
                    else:
                        results = None
                    
                    time.sleep(0.5) 
                
                print(f"      -> Añadidas {songs_added_in_term} canciones de '{term}'.")
                
            except Exception as e:
                print(f"   -> Error al buscar con término '{term}': {e}")
                time.sleep(2) 
            
    return pd.DataFrame(spotify_data)

In [53]:
# --- LLAMADA AL CÓDIGO ---

if sp:
    # Pasamos la lista de términos como argumento
    df_spotify_raw = extract_spotify_data_reggaeton_ampliada(sp, YEARS, query_terms, max_total_songs=2000)
    print("\n--- RESULTADO FINAL DE REGGAETON (ESTRATEGIA ROBUSTA) ---")
    print(f"Total de registros extraídos: {len(df_spotify_raw)} (Nota: este número ya elimina duplicados entre términos)")
    print(df_spotify_raw.head(10))


--- INICIANDO EXTRACCIÓN PARA EL AÑO 2000 ---
   -> Buscando con término: reggaeton
   -> Spotify reporta 60 resultados para 'reggaeton'.
      -> Añadidas 60 canciones de 'reggaeton'.
   -> Buscando con término: latin urban
   -> Spotify reporta 47 resultados para 'latin urban'.
      -> Añadidas 45 canciones de 'latin urban'.
   -> Buscando con término: pop latino
   -> Spotify reporta 100 resultados para 'pop latino'.
      -> Añadidas 100 canciones de 'pop latino'.
   -> Buscando con término: trap latino
   -> Spotify reporta 0 resultados para 'trap latino'.
      -> Añadidas 0 canciones de 'trap latino'.
   -> Buscando con término: urbano latino
   -> Spotify reporta 3 resultados para 'urbano latino'.
      -> Añadidas 0 canciones de 'urbano latino'.
   -> Buscando con término: dembow
   -> Spotify reporta 1 resultados para 'dembow'.
      -> Añadidas 1 canciones de 'dembow'.
   -> Buscando con término: "hip hop latino"
   -> Spotify reporta 29 resultados para '"hip hop latino"'.

In [54]:
# --- NUEVA FUNCIÓN: EXTRACCIÓN DE DETALLES DEL ÁLBUM ---

def extract_album_details(sp, album_ids):
    """
    Extrae los metadatos completos (nombre, año, artista) de los álbumes.
    """
    album_details = []
    
    # Usamos un set para IDs de álbumes ya procesados
    unique_album_ids = set(album_ids)
    
    print(f"\n--- INICIANDO EXTRACCIÓN DE METADATOS DE {len(unique_album_ids)} ÁLBUMES ---")
    
    for album_id in unique_album_ids:
        try:
            # Endpoint de Spotify para obtener detalles de un álbum específico
            album_info = sp.album(album_id) 
            
            data_row = {
                'ID_Album': album_id,
                'Nombre_Album': album_info['name'],
                'Año_Lanzamiento_Album': album_info['release_date'][:4], # Tomamos solo el año
                'Artista_Principal_Album': album_info['artists'][0]['name'],
                'Tipo_Lanzamiento': album_info['album_type'] # (album, single, compilation)
            }
            album_details.append(data_row)
            time.sleep(0.2) # Pausa ligera
            
        except Exception as e:
            # Imprime un mensaje si un álbum falla (ej. si fue eliminado de Spotify)
            print(f"   -> Error al extraer detalles del álbum {album_id}: {e}")
            pass
            
    print(f"✅ Extracción de {len(album_details)} álbumes completada.")
    return pd.DataFrame(album_details)

In [55]:
def extract_lastfm_artist_details(artist_name):
    """Extrae biografía y estadísticas de popularidad de un artista."""
    
    url = 'http://ws.audioscrobbler.com/2.0/'
    params = {
        'method': 'artist.getinfo',
        'artist': artist_name,
        'api_key': LASTFM_API_KEY,
        'format': 'json'
    }
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        
        if 'artist' in data:
            artist_info = data['artist']
            
            # Extraer los datos relevantes (Biografía, Popularidad/Plays)
            biography = artist_info.get('bio', {}).get('summary', 'N/A')
            playcount = artist_info.get('stats', {}).get('playcount', '0')
            listeners = artist_info.get('stats', {}).get('listeners', '0')
            
            return {
                'Artista': artist_name,
                'Biografia_Resumen': biography,
                'Playcount_LastFM': playcount,
                'Listeners_LastFM': listeners,
            }
    except Exception:
        # En caso de error de conexión o artista no encontrado, devuelve None
        pass 
        
    return None

# --- Ejecución para Artistas Únicos ---

if 'df_spotify_raw' in locals() and not df_spotify_raw.empty:
    
    # Obtener la lista única de artistas para evitar búsquedas repetidas
    unique_artists = df_spotify_raw['Artista'].unique()
    lastfm_results = []
    
    print("\n--- EXTRACCIÓN DETALLES LAST.FM ---")
    
    for artist in unique_artists: 
        details = extract_lastfm_artist_details(artist)
        if details:
            lastfm_results.append(details)
        time.sleep(0.5) # Pausa crucial para no sobrecargar la API

    df_lastfm = pd.DataFrame(lastfm_results)
    
    # 5. UNIFICACIÓN DE DATOS
    # Fusionar los datos de Spotify con los detalles de Last.fm
    df_final_reggaeton = pd.merge(
        df_spotify_raw, 
        df_lastfm[['Artista', 'Playcount_LastFM', 'Listeners_LastFM']],
        on='Artista', 
        how='left' # Usamos left join para mantener todas las canciones de Spotify
    )
    
    print("\n--- DATAFRAME FINAL DE REGGAETON LISTO PARA ALMACENAMIENTO ---")
    print(df_final_reggaeton.head())
    print(f"Total de registros finales para Reggaeton: {len(df_final_reggaeton)}")


--- EXTRACCIÓN DETALLES LAST.FM ---

--- DATAFRAME FINAL DE REGGAETON LISTO PARA ALMACENAMIENTO ---
  Artista                 Género musical     Tipo       Nombre  \
0  Dj Joe  Reggaeton (Origen: reggaeton)  Canción      Orgasmo   
1  Dj Joe  Reggaeton (Origen: reggaeton)  Canción    Mai Grita   
2  Dj Joe  Reggaeton (Origen: reggaeton)  Canción  Que Melones   
3  Jairos  Reggaeton (Origen: reggaeton)  Canción        Siete   
4  Dj Joe  Reggaeton (Origen: reggaeton)  Canción      Mujeres   

   Año de lanzamiento              ID_Spotify                ID_Album  \
0                2000  0E2PAktDgSw1qXEzOwHLMz  5KjlY2iLyL9qaexUbxTGeo   
1                2000  0YxFcX2Rhfp4mOaLFyTXbV  5KjlY2iLyL9qaexUbxTGeo   
2                2000  2XB1118qP1DLyE4deHZrqv  5KjlY2iLyL9qaexUbxTGeo   
3                2000  4wTee4RHe9rkK9fNCCHMi5  5VT7SiRbeHW2ELcs0wOoOj   
4                2000  0rLJYPOKXUe2TfngVgH0eB  5KjlY2iLyL9qaexUbxTGeo   

  Playcount_LastFM Listeners_LastFM  
0           187135       

In [58]:
# --- CELDA DE RECUPERACIÓN DE LA FUSIÓN (RÁPIDA Y CORREGIDA) ---

if 'df_spotify_final' in locals() and 'df_lastfm' in locals():
    
    # Lista de columnas a fusionar (EXCLUYENDO 'Artistas_Similares')
    lastfm_cols_to_merge = ['Artista', 'Biografia_Resumen', 'Playcount_LastFM', 'Listeners_LastFM']
    
    # 2. Re-ejecutar el merge (fusión)
    df_proyecto_final = pd.merge(
        df_spotify_final, 
        # Seleccionamos solo las columnas que sabemos que existen:
        df_lastfm[lastfm_cols_to_merge], 
        on='Artista', 
        how='left'
    )
    
    print("✅ Fusión final re-ejecutada con éxito. Columnas de popularidad añadidas.")
    
    # 3. Guardar el archivo CSV (Tu Celda 5 original)
    output_filename = 'reggaeton_data_2000_2024.csv'
    
    df_proyecto_final.to_csv(
        output_filename, 
        index=False,        
        encoding='utf-8'    
    )
    
    print(f"\n🎉 ¡FASE 1 COMPLETADA! 🎉")
    print(f"Los datos de {len(df_proyecto_final)} canciones únicas de Reggaeton han sido guardados en: {output_filename}")

else:
    print("❌ ERROR: Las variables df_spotify_final o df_lastfm NO están en la memoria. Por favor, reinicia desde la Celda 3.")

✅ Fusión final re-ejecutada con éxito. Columnas de popularidad añadidas.

🎉 ¡FASE 1 COMPLETADA! 🎉
Los datos de 769 canciones únicas de Reggaeton han sido guardados en: reggaeton_data_2000_2024.csv
